In [2]:
import os
import pandas as pd
from geo import get_place_list, get_place_data
from params import DATA_DIR

Load some data

* `all_geos` is a list of all geographies mentioned in the the geography tree csv file
* `population_data` is population estimates for a (some of) the geographies
* `council_tax_data` is estimates of people in receipt of council tax support
* `clif_data` is data bout children living in poverty 

In [3]:
population_data = pd.read_csv(f'{DATA_DIR}/population-estimates/population-estimates.csv')
population_data = population_data[population_data.age_name == "All Ages"]
#population_data = population_data
population_data = population_data[population_data.date == max(population_data['date'])]
population_data

,date,geography_code,geography_name,geography_type,gender_code,gender_name,age_code,age_name,variable_code,variable_name,value
12,2021,E12999901,The North,panrgn22,0,Total,200,All Ages,n_persons,Number of persons,15550498
18,2021,E12000001,North East,rgn22,0,Total,200,All Ages,n_persons,Number of persons,2646772
36,2021,E12000002,North West,rgn22,0,Total,200,All Ages,n_persons,Number of persons,7422295
54,2021,E12000003,Yorkshire and The Humber,rgn22,0,Total,200,All Ages,n_persons,Number of persons,5481431
72,2021,E12000004,East Midlands,rgn22,0,Total,200,All Ages,n_persons,Number of persons,4880094
...,...,...,...,...,...,...,...,...,...,...,...
6264,2021,E47000007,West Midlands,cauth22,0,Total,200,All Ages,n_persons,Number of persons,2916132
6282,2021,E47000008,Cambridgeshire and Peterborough,cauth22,0,Total,200,All Ages,n_persons,Number of persons,896756
6300,2021,E47000009,West of England,cauth22,0,Total,200,All Ages,n_persons,Number of persons,954276
6318,2021,E47000010,North East,cauth22,0,Total,200,All Ages,n_persons,Number of persons,1139626


In [4]:
gva = pd.read_csv(f'{DATA_DIR}/gva/gva.csv')

In [5]:
area_of_place = pd.read_csv(f'{DATA_DIR}/geo/area_of_places.csv')
#@TODO temporary filter to remove
area_of_place.drop_duplicates(subset=['geography_code'], inplace=True)
# len(area_of_place.geography_code)
# l1 = area_of_place.geography_code.to_list()
# l2 = area_of_place.geography_code.unique()
# seen = set()
# dupes = [x for x in l1 if x in seen or seen.add(x)]
# print(dupes)
area_of_place

,geography_code,variable_name,value
0,E05000650,Area in sq km,6.556496
1,E05000651,Area in sq km,8.998946
2,E05000652,Area in sq km,3.719582
3,E05000653,Area in sq km,7.340085
4,E05000654,Area in sq km,3.497711
...,...,...,...
1646,E11000007,Area in sq km,538.992682
1647,E12000001,Area in sq km,8563.803423
1648,E12000002,Area in sq km,14103.955722
1649,E12000003,Area in sq km,15369.599209


In [6]:
households = pd.read_csv(f'{DATA_DIR}/households/households.csv')

In [7]:
council_tax_data = pd.read_csv(f'{DATA_DIR}/council-tax-support/council-tax-support.csv')
council_tax_data = council_tax_data[council_tax_data.date == max(council_tax_data.date)]

In [8]:
clif_data = pd.read_csv(f'{DATA_DIR}/clif/clif_REL.csv')
clif_data = clif_data[
    (clif_data['Age of Child (years and bands)'] == 'Total') &
    (clif_data['Gender of Child'] == 'Total') &
    (clif_data['Family Type'] == 'Total') &
    (clif_data['Work Status'] == 'Total') &
    (clif_data.Year == max(clif_data.Year))
]
clif_data

,Year,Age of Child (years and bands),Gender of Child,Family Type,Work Status,variable_name,geography_code,value
134,2021/22,Total,Total,Total,Total,children_in_low_income,E06000047,27178.0
296,2021/22,Total,Total,Total,Total,children_in_low_income,E06000005,6070.0
458,2021/22,Total,Total,Total,Total,children_in_low_income,E06000001,5870.0
620,2021/22,Total,Total,Total,Total,children_in_low_income,E06000002,13432.0
782,2021/22,Total,Total,Total,Total,children_in_low_income,E06000057,13562.0
...,...,...,...,...,...,...,...,...
60236,2021/22,Total,Total,Total,Total,children_in_low_income,Mid and East Antrim,5687.0
60398,2021/22,Total,Total,Total,Total,children_in_low_income,Mid Ulster,8079.0
60560,2021/22,Total,Total,Total,Total,children_in_low_income,"Newry, Mourne and Down",10990.0
60722,2021/22,Total,Total,Total,Total,children_in_low_income,Unknown,0.0


In [9]:
number_of_persons = pd.read_csv(f'{DATA_DIR}/population-estimates/population-estimates-ages.csv')
number_of_children = number_of_persons[number_of_persons.age_band.isin(['0-4', '5-10', '11-15', '16-19'])]
number_of_children = number_of_children.groupby('geography_code').sum(numeric_only=True).reset_index()
number_of_children['variable_name'] = 'number_of_children'
number_of_children

,geography_code,date,value,variable_name
0,E05000650,8080,3390,number_of_children
1,E05000651,8080,2530,number_of_children
2,E05000652,8080,3722,number_of_children
3,E05000653,8080,2813,number_of_children
4,E05000654,8080,4875,number_of_children
...,...,...,...,...
1315,E47000003,8084,586078,number_of_children
1316,E47000004,8084,346827,number_of_children
1317,E47000006,8084,159599,number_of_children
1318,E47000010,8084,247569,number_of_children


In [17]:
#number of households in poverty
households_in_poverty = pd.read_csv(f'{DATA_DIR}/hbai/Type of Individual by Age Category.csv')
households_in_poverty = households_in_poverty[households_in_poverty.variable_name == "In low income (below threshold)"]

def three_year_average(data):
    #@TODO function will need work to account for 3 year average of any period given.
    #filtering out 2020-21 as no data was collected
    data = data[data["Financial Year"] != "2020-21"]
    #getting a list of dates
    dates = data["Financial Year"].unique()
    most_recent_dates = list(dates[-2:])
    #print(most_recent)
    data = data[data["Financial Year"].isin(most_recent_dates)]
    #calculating a 2 year average according to user guidance
    data = data.groupby('geography_code').sum(numeric_only=True) / len(most_recent_dates)
    #rounding to nearest 0.1mil, according to user guidance.
    data = data.round(-5).reset_index()
    return data

households_in_poverty = three_year_average(households_in_poverty)

households_in_poverty['variable_name'] = 'Households in poverty'
households_in_poverty

,geography_code,value,variable_name
0,E12000001,500000.0,Households in poverty
1,E12000002,1300000.0,Households in poverty
2,E12000003,1100000.0,Households in poverty


In [18]:
savings = pd.read_csv(f'{DATA_DIR}/hbai/Savings and Investments of Adults in the Family of the Individual.csv')
savings = savings[savings['Financial Year'] == max(savings['Financial Year'])]
savings = savings[savings.variable_name == 'In low income (below threshold)']
savings = savings[savings['Savings and Investments of Adults in the Family of the Individual'] == 'No savings']
savings.variable_name = 'households_low_income_no_savings'
savings = three_year_average(savings)
savings
#save each geog_code into 'src/place/EXXXXXX/_data/' and then can build the visualisations in place.njk on a conidtional statement if that data exists.

,geography_code,value
0,E12000001,100000.0
1,E12000002,400000.0
2,E12000003,300000.0


In [11]:
 # Index of Multiple deprivation
data = pd.read_csv(f'{DATA_DIR}/imd/imd.csv')
data  = data[data.variable_name == 'Average score'] #adding this as in future there will be measures for each age cat.#
imd = data[data.dataset == 'IMD']

imd_older_people = data.loc[data.dataset == 'IDAOPI'].copy()
imd_older_people['variable_name'] = 'imd_older_people'

imd_children = data.loc[data.dataset == "IDACI"].copy()
imd_children['variable_name'] = 'imd_children'


In [12]:
labour_market = pd.read_csv(f'{DATA_DIR}/labour-market/labour-market.csv')
unemployment = labour_market[labour_market.variable_name == "Unemployment rate - aged 16-64"]
unemployment = unemployment[unemployment.date == max(unemployment.date)]
unemployment['variable_name'] = 'unemployment_rate_16_64'

labour_market = pd.read_csv(f'{DATA_DIR}/labour-market/labour-market.csv')
economic_inactivity = labour_market[labour_market.variable_name == "% who are economically inactive - aged 16-64"]
economic_inactivity = economic_inactivity[economic_inactivity.date == max(economic_inactivity.date)]
economic_inactivity = economic_inactivity.replace("% who are economically inactive - aged 16-64","economic_inactivity_16_64")

economic_inactivity

,date,geography_code,geography_name,variable_code,variable_name,value
2,2022-12-01,E06000005,Darlington,111,economic_inactivity_16_64,20.9
5,2022-12-01,E06000047,County Durham,111,economic_inactivity_16_64,24.3
8,2022-12-01,E06000001,Hartlepool,111,economic_inactivity_16_64,27.3
11,2022-12-01,E06000002,Middlesbrough,111,economic_inactivity_16_64,29.3
14,2022-12-01,E06000057,Northumberland,111,economic_inactivity_16_64,25.8
...,...,...,...,...,...,...
221,2022-12-01,E10000017,Lancashire,111,economic_inactivity_16_64,23.5
224,2022-12-01,E10000023,North Yorkshire,111,economic_inactivity_16_64,20.0
227,2022-12-01,E12000001,North East,111,economic_inactivity_16_64,26.0
230,2022-12-01,E12000002,North West,111,economic_inactivity_16_64,23.3


Concatenate the loaded data, then pivot into a table with a line per geography code. Filter this to only include geographies that are in the canonical list of areas.

In [15]:
place_data = pd.concat([
    population_data,
    area_of_place,
    gva,
    households,
    council_tax_data,
    clif_data,
    number_of_children,
    savings,
    imd,
    households_in_poverty,
    unemployment,
    economic_inactivity,
    imd_children,
    imd_older_people
]).pivot(index='geography_code', columns='variable_name', values='value')
place_data = place_data.loc[place_data.index.isin(get_place_list())]
place_data = place_data.merge(get_place_data(), left_index=True, right_index=True, how='outer')
place_data.index.name = 'geography_code'
place_data

,Area in sq km,Average score,GVA,Households in poverty,Number of households,Number of persons,children_in_low_income,economic_inactivity_16_64,households_low_income_no_savings,imd_children,imd_older_people,number_of_children,pensioners,unemployment_rate_16_64,working_age,ancestors,parents,children,name,type
geography_code,,,,,,,,,,,,,,,,,,,,
E05000650,6.556496,NaN,82.047782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3390.0,NaN,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Astley Bridge,WD22
E05000651,8.998946,NaN,79.731727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2530.0,NaN,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Bradshaw,WD22
E05000652,3.719582,NaN,178.319099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3722.0,NaN,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Breightmet,WD22
E05000653,7.340085,NaN,110.950950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2813.0,NaN,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Bromley Cross,WD22
E05000654,3.497711,NaN,275.878141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4875.0,NaN,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Crompton,WD22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E47000003,2023.378010,NaN,57909.000000,NaN,1035190.0,2349987.0,NaN,NaN,NaN,NaN,NaN,586078.0,NaN,NaN,NaN,"[E12000003, E12999901]",[E12000003],"[E08000032, E08000033, E08000034, E08000035, E...",West Yorkshire,CAUTH22
E47000004,723.925858,NaN,33598.000000,NaN,729960.0,1551722.0,NaN,NaN,NaN,NaN,NaN,346827.0,NaN,NaN,NaN,"[E12000002, E12999901]","[E12000002, E12000002]","[E08000011, E08000012, E08000013, E08000014, E...",Liverpool City Region,CAUTH22
E47000006,792.415536,NaN,13951.000000,NaN,317080.0,678173.0,NaN,NaN,NaN,NaN,NaN,159599.0,NaN,NaN,NaN,"[E12000001, E12999901]",[E12000001],"[E06000001, E06000002, E06000003, E06000004, E...",Tees Valley,CAUTH22


Create some additional metrics based on the data in place_data

In [16]:
place_data['Population density'] = place_data['Number of persons'] / place_data['Area in sq km']

Finally, write the data to an interim parquet and json file for later usage.

In [ ]:
os.makedirs(f'{DATA_DIR}/interim/', exist_ok=True)
place_data.to_parquet(f'{DATA_DIR}/interim/place_data.parquet')
place_data.reset_index().to_json(f"{DATA_DIR}/interim/place_data.json", orient="records")